# 集成学习 ensemble learning

集成学习（ensemble learning）可以说是现在非常火爆的机器学习方法了。  
它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务。  
也就是我们常说的“博采众长”。  

集成学习可以用于
- 分类问题集成，
- 回归问题集成，
- 特征选取集成，
- 异常点检测集成等等，   
可以说所有的机器学习领域都可以看到集成学习的身影。  


## 1.集成学习概述
从下图，我们可以对集成学习的思想做一个概括。  
对于训练数据集，我们通过若干个个体学习器，通过一定的结合策略，就可以最终形成一个强学习器，以达到博采众长的目的。  

![title](../images/ensemble_001.png)

也就是说，集成学习有两个主要问题需要解决：
- 1. 如何得到若干个个体学习器
- 2. 如何选择